In [1]:
import random
import time
import numpy as np
# Import all modules for RF (RAPIDS, DASK, etc)
from load_dataset import custom_lakedepth as ld
from pprint import pprint

from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from cuml.dask.common import utils as dask_utils
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import dask_cudf

from cuml.dask.ensemble import RandomForestRegressor as cumlDaskRF

In [2]:
# This will use all GPUs on the local host by default
cluster = LocalCUDACluster(threads_per_worker=1)
c = Client(cluster)

# Query the client for all connected workers
workers = c.has_what().keys()
n_workers = len(workers)
n_streams = 8 # Performance optimization

c

data = ld.LakeDepth(42)
covariates_train, covariates_test, labels_train, labels_test = data.split(0.20)
covariates_test_pd = covariates_test.to_pandas()
labels_test_pd = labels_test.to_pandas()

/home/cssprad1/.conda/envs/myRapids/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38099 instead
  http_address["port"], self.http_server.port


/att/gpfsfs/briskfs01/ppl/cssprad1/projects/rapids_rf_lake_depth/load_dataset/LakeDepth/pts_merged_final.csv
 - from DATA: reading csv into GPU memory
 - from DATA: done reading csv into GPU memory
 - from DATA: dropped column: FID
 - from DATA: dropped column: Date


In [3]:
n_partitions = n_workers

In [4]:
k_fold = KFold(5)
results = []
TEST_SIZE = 0.20
RANDOM_STATE = 42
# Introduce lists to the hyperparameters we want to

N_ESTIMATORS = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
SPLIT_ALGO = 1
SPLIT_CRITERION = 2
BOOTSTRAP = [True, False]
BOOTSTRAP_FEATURES = False
ROWS_SAMPLE = 1.0
MAX_DEPTH = [int(x) for x in np.linspace(10, 110, num=11)]
MAX_LEAVES = -1
MAX_FEATURES = ['auto', 'sqrt']
N_BINS = [int(x) for x in np.linspace(start=5, stop=20, num=10)]
MIN_ROWS_PER_NODE = 2
MIN_IMPURITY_DECREASE = 0.0
ACCURACY_METRIC = 'mean_ae'  # 'mse' #'r2' # 'median_aw' #
QUANTILEPT = False
SEED = 42
VERBOSE = False

random_grid = {'n_estimators': N_ESTIMATORS,
               'max_depth': MAX_DEPTH,
               'bootstrap': BOOTSTRAP,
               'max_features': MAX_FEATURES,
               'n_bins': N_BINS}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
 'max_features': ['auto', 'sqrt'],
 'n_bins': [5, 6, 8, 10, 11, 13, 15, 16, 18, 20],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [5]:
for i in range(5):
    print("  - from Random Search: Epoch #", i)
    depth_rf_model = cumlDaskRF(n_estimators=random.choice(N_ESTIMATORS),
                          split_algo=SPLIT_ALGO,
                          split_criterion=SPLIT_CRITERION,
                          bootstrap=random.choice(BOOTSTRAP),
                          bootstrap_features=BOOTSTRAP_FEATURES,
                          rows_sample=ROWS_SAMPLE,
                          max_depth=random.choice(MAX_DEPTH),
                          max_leaves=MAX_LEAVES,
                          max_features=random.choice(MAX_FEATURES),
                          n_bins=random.choice(N_BINS),
                          min_rows_per_node=MIN_ROWS_PER_NODE,
                          min_impurity_decrease=MIN_IMPURITY_DECREASE,
                          accuracy_metric=ACCURACY_METRIC,
                          quantile_per_tree=QUANTILEPT,
                          seed=SEED,
                          verbose=VERBOSE)

    scores = []
    st = time.time()

    for k, (train, test) in enumerate(k_fold.split(covariates_train, labels_train)):
       
        X_dask_cudf = dask_cudf.from_cudf(covariates_train.iloc[train], npartitions=n_workers)
        y_dask_cudf = dask_cudf.from_cudf(labels_train.iloc[train], npartitions=n_workers)
        X_dask_cudf, y_dask_cudf = dask_utils.persist_across_workers(c,
                                                  [X_dask_cudf,
                                                   y_dask_cudf])
        depth_rf_model.fit(X_dask_cudf, y_dask_cudf)
        wait(depth_rf_model.rfs)
        print("Starting predictions")
        predictions = depth_rf_model.predict(labels_train.to_pandas())
        score = mean_absolute_error(labels_test_pd.to_pandas(), pred.to_pandas())
        # score = depth_rf_model.score(covariates.iloc[test],
        # labels.iloc[test])
        scores.append(score)

        et = time.time()
        print("   -time to train (sec): ", et-st)

    results.append({'n_estimators': depth_rf_model.n_estimators,
                    'bootstrap': depth_rf_model.bootstrap,
                    'max_depth': depth_rf_model.max_depth,
                    'max_features': depth_rf_model.max_features,
                    'n_bins': depth_rf_model.n_bins,
                    'performance': np.mean(scores)})

  - from Random Search: Epoch # 0
Starting predictions


ValueError: Cannot create cython.array from NULL pointer